# Human numbers

In [2]:
from fastai.text import *

In [3]:
bs=64

## Data

Untar the data at `URLs.HUMAN_NUMBERS` and save the path to `path`. Show the files in `path`.

Define a function `readnums` that takes in an argument `d` and returns the lines in `path/d` separated by ', '.

Use `readnums` to read `train.txt` and show the first line and the first 80 values, saving it to `train_txt`.

Do the same for the validation, only this time we'll look for the _last_ 80 values in the first row. Save it to `valid_txt`.

Create a `TextList` from each of `train_txt` and `val_txt`, using `path`. Create `src`, an `ItemLists`, from `path`, `train`, and `valid` (the two `TextList`s that you just created). Label it for a language model (see `label_for_lm`). Create a databunch where batch size is equal to `bs` (defined above).

Index into the training data and visualize it using the `.text` property. Show the first 80 characters of the first example in the training set.

Show the length of the `x` portion of the first example in the validation set. Hint: check out the property `data`. On the example. What does this do?

Show a tuple of the `data.bptt` property, and the length of `data.valid_dl`. Why is the length of the valid_dl what it is? 

Divide the length of the first validation example by the bptt size and it should equal a little less than the length of the validation data loader.

Get an iterator of the `valid_dl` and call it `it`. Call `next` and set it to `x1,y1`. Do it again and call it `x2,y2`. Again and call it `x3,y3`. Then close the iterator.

Show the textified version of `x1[0]`, and then the textified version of `y1[0]`. 

Do the same for `x2[0]` and `y2[0]`.

Now do `x1[1]` and `y1[1]`.

Show the shape of x1.

What have you noticed about the structure of the data?

Show the total number of elements in the whole dataset.

Show the shapes of x1 and y1.

Show the shapes of x2 and y2.

Show `x1[0]` and `y1[0]`. What do you notice about them?

Before each of the following, try to take a guess at which data is going to be captured. It might be helpful to have a way to draw it out on the side.

Textify the x1[0] and y1[0].

Set `v` to `data.valid_ds.vocab`.

Textify `x1[0]`.

Textify `y1[0]`.

Textify `x2[0]`.

Texxtify `x3[0]`.

Textify `x1[1]`.

Textify `x2[1]`.

Textify `x3[1]`.

Textify the last example in `x3`.

Show a batch from the validation set.

## Single fully connected model

Set `data` to a databunch from `src` with `bs` from above, abd `bptt=3`.

Set `x,y` to the result of a batch and check their shapes.

Set `nv` to the length of `v.itos`. What is `v.itos`? Print the value.

Set `nh` to 64. Can you guess what `nh` is for?

Define a function `loss4` that takes an input and a target and calls `F.cross_entropy` on input and the last column of the target. Similarly do do the same thing for `accuracy` which calls `accuracy` on the last column. Why do we care about the last column?

Create a class `Model0` the inherits from `nn.Module`. Make sure you call `super.__init__()`. Then, set four layers at the instance level:
- self.i_h should be an embedding layer (nv, nh) 
- self.h_h should be a linear layer (nh, nh) 
- self.h_o should be a linear layer (nh, nv) 
- self.bn should be a batchnorm layer (nh)

`h` should start by calling i_h on the x inputs, along with a relu and a batchnorm. Then, if x.shape[1] > 1, we'll reset `h` as the sum of itself and i_h of x[:,1]. We'll then set it again as the batch-normed relu'd h_h of itself. If x.shape [1] > 2, we'll do this again with x[:,2]. Finally, we'll return h_o of the hidden layer. Can you explain how this works (even if at a high level)?

Set `learn` to a `Learner` using `data` and a `Model0`, with loss func `loss4` and metrics `acc4`.

Fit a cycle with 6 epochs at a learning rate of `1e-4`.

## Same thing with a loop

Now write the same thing as above with a loop. Call the class `Model1`. Hint: to initialize a torch zero vector on the GPU, you'll have to call `.to(device=x.device)`.

## Multi fully connected model

Create a new `data` that has `bptt=20`.

Grab a batch and set it to `x,y`. Show the shapes.

Write a new module `Model2` that accumulates all of the calls to `h_o` in a list called `res`, and returns a torch-stacked (across the horizontal axis) collection of all of the values in `res`. What would you expect this output to look like? What would you expect the benefit of doing this to be?

Create a learner with `data` and `Model2()` with `metrics=accuracy`.

Fit a cycle at 1e-4 and `pct_start=0.1`.

The loss is worse; why do you think this is?

## Maintain state

Write a class that's similar to the ones above except: 
- In __init__ we'll set a zero-matrix of size bs x nh (remember to cuda it; what does this do?)
- Instead of initializing `h` to a zero-vector in `forward`, we'll set it to self.h
- After the loop. we'll call `self.h = h.detach()`; what does `detach` do?

What's the benefit of doing this instead of what we did above?

Create a learner with `data`, `Model3()`, and `metrics=accuracy`.

Fit a 20-epoch cycle with an lr of 3e-3.

## nn.RNN

Now we'll use the rnn abstraction just to see how that works. The idea is simple -- we do what we did for the "maintain state" edition, except we have a new instance level object `self.rnn` which is set to `nn.RNN(nh,nh,batch_first=True)`, and our entire loop is replaced with a call to `self.rnn(self.i_h(x), self.h)`, which returns `res,h`. We still need to detach the hidden layer to prevent backprop from changing it.

Create a learner with `data`, `Model4()`, and `metrics=accuracy`.

Fit a 20-epoch cycle with an lr of 3e-3.

## 2-layer GRU

To create a 2-layer GRU we'll do the same, except `self.rnn = nn.GRU(nh, nh, 2, batch_first=True)`. 

Create a similar learner.

Fit a 10-epoch cycle with lr 1e-2.

## fin